# Data Cleaning Project

## Cafe Sales

This project will show how I go about injesting, investigating and cleaning dirty data. The finished product will be a clean data set that can be used for futher analysis or other data projects. The data used is the cafe sales data dowloaded from Kaggle at the following location: https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training

For the purposes of this exercise, I have decided that the cafe wants to understand which location is the most lucrative (how much is spent in each location) and which products are most popular (how many are bought in each location) in each location. Date isn't necessarily important for the moment, however it may be down the line. This will guide further analysis in the future, however for this excercise, it will help us understand which parts of the data are important and which aren't.

### 1.0 Data Injestion & Investigation

In [1]:
# Importing the necessary libraries - these are the standard libraries that I tend to import for all projects regardless of what I'm doing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [2]:
# Importing the data set and saving as a DataFrame that we wont touch
orig_sales = pd.read_csv('cafe_sales_data.csv')

# Creating a copy of the DataFrame to work with
sales = orig_sales.copy()

# Viewd the DataFrame
sales.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


Straight out of the gate we can see some issues with the data, in both payment method and location there are 'UNKNOWN' values and we have an 'ERROR' in the total spend column. I will have to dive a little deeper to understand how many occurences there are of each of these and if there are any other anomalies. 

In [3]:
# Check the metadata of the DataFrame
sales.info()

# Check the shape of the DataFrame
sales.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


(10000, 8)

In [4]:
# Checking the data types
sales.dtypes

Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object

Again, we can see there are some formatting errors. The obvious one is the date column which is currently an object. We would like this to be a datetime data type.

In [5]:
# Checking for null values in the DataFrame
sales.isnull().sum()


Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

We can see that there are nulls in every column. Now there are many ways we can deal with these, particularly in the numerical columns. I think we have to understand the data a little better to make a decision on what to do with these nulls.

In [6]:
# Checking for dup;icate values
sales.duplicated().sum()

0

No duplicates, thats a positive.

In [7]:
# Finally we'll look at some summary statistics of the data
sales.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


Not massively helpful or insightful, but gives us a visualisation of the number of data points in each column.

### 2.0 Data Cleaning

In [8]:
# Renaming the columns to make the data easier to work with
sales.rename(columns={'Transaction ID': 'transaction_id',
                 'Item': 'item',
                 'Quantity': 'quantity',
                 'Price Per Unit': 'price',
                 'Total Spent': 'spent',
                 'Payment Method': 'pay_method',
                 'Location': 'location',
                 'Transaction Date': 'date'}, inplace=True)

# check the columns
sales.columns

Index(['transaction_id', 'item', 'quantity', 'price', 'spent', 'pay_method',
       'location', 'date'],
      dtype='object')

In [9]:
# Check the number of unique values in each column
sales.nunique()

transaction_id    10000
item                 10
quantity              7
price                 8
spent                19
pay_method            5
location              4
date                367
dtype: int64

### 2.1: Column by column cleaning
I'll go through each column that will be important to our analysis and correct any issues, change data types if necessary and/or remove any erroneous entries.

#### 2.1.1: Items & Price

In [10]:
# Checking each of the individual items
sales['item'].unique()



array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'UNKNOWN',
       'Sandwich', nan, 'ERROR', 'Juice', 'Tea'], dtype=object)

We can see there some erroneous entries in the item column, but using the price column, we may be able to workout what these erroneous entries are (assuming that each item has a unique price)

In [11]:
# Check the unique price points of the items
sales.groupby('item')['price'].unique()

item
Cake                               [3.0, nan, UNKNOWN, ERROR]
Coffee                             [2.0, nan, ERROR, UNKNOWN]
Cookie                             [1.0, UNKNOWN, nan, ERROR]
ERROR       [1.5, 3.0, 5.0, nan, 4.0, 2.0, 1.0, UNKNOWN, E...
Juice                              [3.0, nan, UNKNOWN, ERROR]
Salad                              [5.0, ERROR, UNKNOWN, nan]
Sandwich                           [4.0, nan, ERROR, UNKNOWN]
Smoothie                           [4.0, nan, UNKNOWN, ERROR]
Tea                                [1.5, nan, ERROR, UNKNOWN]
UNKNOWN     [3.0, 1.0, 5.0, 4.0, 1.5, 2.0, nan, UNKNOWN, E...
Name: price, dtype: object

In [12]:
# Replacing the error, unknown and nan values in price with the correct values
sales.loc[sales['item'] == 'Cake', 'price'] = '3.0'
sales.loc[sales['item'] == 'Coffee', 'price'] = '2.0'
sales.loc[sales['item'] == 'Cookie', 'price'] = '1.0'
sales.loc[sales['item'] == 'Juice', 'price'] = '3.0'
sales.loc[sales['item'] == 'Salad', 'price'] = '5.0'
sales.loc[sales['item'] == 'Sandwich', 'price'] = '4.0'
sales.loc[sales['item'] == 'Smoothie', 'price'] = '4.0'
sales.loc[sales['item'] == 'Tea', 'price'] = '1.5'

# Replacing the error, unknown and nan values in item with the correct values
sales.loc[sales['price'] == '2.0', 'item'] = 'Coffee'
sales.loc[sales['price'] == '1.0', 'item'] = 'Cookie'
sales.loc[sales['price'] == '5.0', 'item'] = 'Salad'
sales.loc[sales['price'] == '1.5', 'item'] = 'Tea'


# Check the unique price points of the items again
sales.groupby('item')['price'].unique()

item
Cake                                  [3.0]
Coffee                                [2.0]
Cookie                                [1.0]
ERROR       [3.0, nan, 4.0, UNKNOWN, ERROR]
Juice                                 [3.0]
Salad                                 [5.0]
Sandwich                              [4.0]
Smoothie                              [4.0]
Tea                                   [1.5]
UNKNOWN     [3.0, 4.0, nan, UNKNOWN, ERROR]
Name: price, dtype: object

We did what we could with the information we had. Now the best thing to do is to remove the errors, unknowns and nan values from the item list and continue our data cleaning.

In [13]:
# Remove the error, unknown and nan values in price and item
sales = sales[~sales['item'].str.contains('UNKNOWN|ERROR', na=False)]
sales = sales[~sales['price'].str.contains('UNKNOWN|ERROR', na=False)]
sales = sales[~sales['item'].isnull()]

# Check the number unique price points for the items
sales.groupby('item')['price'].nunique()

item
Cake        1
Coffee      1
Cookie      1
Juice       1
Salad       1
Sandwich    1
Smoothie    1
Tea         1
Name: price, dtype: int64

In [14]:
# Change the data type of the price column to float
sales['price'] = sales['price'].astype(float)

# Check the data types of the DataFrame again
sales.dtypes

transaction_id     object
item               object
quantity           object
price             float64
spent              object
pay_method         object
location           object
date               object
dtype: object

That's the item & price columns cleaned, lets continue.

#### 2.1.2: Quantity

As this is a case by case data series, there really isn't much we can do other than drop any erroneous entries and change the data type

In [15]:
# Check the unique values in the quantity column
sales['quantity'].unique()

array(['2', '4', '5', '3', '1', 'ERROR', 'UNKNOWN', nan], dtype=object)

In [16]:
# Drop the ERROR, UNKOWN and Nan values in the quantity column
sales = sales[~sales['quantity'].str.contains('UNKNOWN|ERROR', na=False)]
sales = sales[~sales['quantity'].isnull()]

# Check the unique values in the quantity column again
sales['quantity'].unique()

array(['2', '4', '5', '3', '1'], dtype=object)

In [17]:
# Change the data type of quantity to int
sales['quantity'] = sales['quantity'].astype(int)

# Check the data types of the DataFrame again
sales.dtypes

transaction_id     object
item               object
quantity            int32
price             float64
spent              object
pay_method         object
location           object
date               object
dtype: object

#### 2.1.3: Spent
Again, this is a case by case series, however where we have both quantity and price (which we do because we've cleaned both of those columns) we can deduct a quantity spent for any erroneous entries.

In [18]:
# Check the unique values in the quantity column
sales['spent'].unique()

array(['4.0', '12.0', 'ERROR', '10.0', '20.0', '16.0', '25.0', '8.0',
       '5.0', '3.0', '15.0', '6.0', nan, 'UNKNOWN', '2.0', '9.0', '1.0',
       '7.5', '4.5', '1.5'], dtype=object)

In [19]:
# Multiply price by quantity where spent contains ERROR, UNKNOWN or nan values
sales.loc[sales['spent'].str.contains('UNKNOWN|ERROR', na=False), 'spent'] = sales['price'] * sales['quantity']
sales.loc[sales['spent'].isnull(), 'spent'] = sales['price'] * sales['quantity']

# Check the unique values in the spent column again
sales['spent'].unique()

array(['4.0', '12.0', 4.0, '10.0', '20.0', '16.0', '25.0', '8.0', '5.0',
       '3.0', '15.0', '6.0', 12.0, 2.0, 3.0, '2.0', '9.0', '1.0', '7.5',
       '4.5', 9.0, '1.5', 6.0, 20.0, 7.5, 15.0, 10.0, 25.0, 8.0, 1.0,
       16.0, 1.5, 4.5, 5.0], dtype=object)

In [20]:
# Change the data type of spent to float
sales['spent'] = sales['spent'].astype(float)

# Check the data types of the DataFrame again
sales.dtypes

transaction_id     object
item               object
quantity            int32
price             float64
spent             float64
pay_method         object
location           object
date               object
dtype: object

#### 2.1.4: Payment Method
The payment method really isn't that important for what it is we're trying to find, so I'll go ahead and drop this column completely

In [21]:
# Drop the payment method column as it is not needed for the analysis
sales.drop(columns=['pay_method'], inplace=True)

# Check the columns of the DataFrame again
sales.columns

Index(['transaction_id', 'item', 'quantity', 'price', 'spent', 'location',
       'date'],
      dtype='object')

#### 2.1.5: Location
Location is a difficult one. It's fundamental to the analysis that we want to do, however it's a case by case data series once again. There are two options, the first it to simply remove the erroneous entries and move on. The second, and the choice I've opted for, is to aggregate the location based on the most common location for each item and replace any erroneous entries with the result. While it may not be the most accurate, it will give us the most amount of data possible for further analysis.

In [22]:
# Check the unique values in the location column
sales['location'].unique()

array(['Takeaway', 'In-store', 'UNKNOWN', nan, 'ERROR'], dtype=object)

In [23]:
# identifying the most common location for each product
sales.groupby('item')['location'].agg(lambda x: x.value_counts().index[0]).reset_index(name='most_common_location')

,item,most_common_location
0,Cake,Takeaway
1,Coffee,Takeaway
2,Cookie,Takeaway
3,Juice,In-store
4,Salad,In-store
5,Sandwich,In-store
6,Smoothie,In-store
7,Tea,In-store


In [24]:
# replacing any UNKNOWN, ERROR and null values in the location column with the most common location for that item
# Calculate the most common location for each item
most_common_locations = sales.groupby('item')['location'].agg(lambda x: x.value_counts().index[0]).to_dict()

# Replace UNKNOWN, ERROR, and NaN values in the location column
sales['location'] = sales.apply(
    lambda row: most_common_locations[row['item']] if pd.isnull(row['location']) or row['location'] in ['UNKNOWN', 'ERROR'] else row['location'],
    axis=1
)

# Check the unique values in the location column again
sales['location'].unique()

array(['Takeaway', 'In-store'], dtype=object)

#### 2.1.6: Date
Whilst the date column isn't explicitly needed, it's important to think ahead about what sort of analysis might be done in the future. We might like to look at the seasonality of sales or understand if the time of year affects the location of the purchase. In that case we will clean up the date column and make it easier to work with in the future.

In [25]:
# Check the unique values in the date column
sales['date'].unique()


array(['2023-09-08', '2023-05-16', '2023-07-19', '2023-04-27',
       '2023-06-11', '2023-03-31', '2023-10-28', '2023-12-31',
       '2023-11-07', 'ERROR', '2023-05-03', '2023-06-01', '2023-03-21',
       '2023-11-15', '2023-06-10', '2023-02-24', '2023-03-25',
       '2023-01-15', '2023-03-30', '2023-12-01', '2023-09-18',
       '2023-06-03', '2023-12-13', '2023-04-20', '2023-04-10',
       '2023-03-11', '2023-06-02', '2023-11-06', '2023-08-15',
       '2023-10-09', '2023-05-28', '2023-04-29', '2023-06-08',
       '2023-06-29', '2023-04-17', '2023-12-22', '2023-01-10',
       '2023-10-02', '2023-02-23', '2023-03-22', '2023-11-03',
       '2023-03-02', '2023-06-26', '2023-05-02', '2023-09-05',
       '2023-01-08', '2023-03-15', '2023-11-25', '2023-12-05',
       '2023-06-27', '2023-10-07', '2023-09-30', '2023-05-27',
       '2023-11-18', '2023-10-20', '2023-10-03', '2023-10-27',
       '2023-01-31', '2023-12-08', '2023-06-19', '2023-12-14',
       '2023-07-16', '2023-02-22', nan, '2023-

As per usual we can see that there are some erroneous entries in the date column. As the date column is important, I'm reluctant to just go ahead and drop the rows. I think the best course of action would be get a feeling for how many erroneous and null values there are and, assuming there isn't too many, generate random dates between the start and end dates and use those to replace the erroneous entries.

In [26]:
# Calculate the number of UNKNOWN, ERROR and null values in the date column
sales['date'].isnull().sum(), sales['date'].str.contains('UNKNOWN|ERROR').sum()

(142, 271)

There isn't too many erroneous values, therefore I think our plan may work. The first thing I need to do is to isolate the erroneous rows, save them to a new data frame and then drop them from the main sales data. This will allow me to change the data type of the date column and find the start and end dates in the valid data. Then I'll generate the random dates in the seperated data and add them back into the main data set.

In [27]:
# Store the unknown, error and null values from the date column in new DataFrames
sales_null = sales[sales['date'].isnull()]
sales_error = sales[sales['date'].str.contains('UNKNOWN|ERROR', na=False)]

# Drop the error, unknown and null values in the date column
sales = sales[~sales['date'].isnull()]
sales = sales[~sales['date'].str.contains('UNKNOWN|ERROR', na=False)]

In [28]:
# Change the data type of the date column to datetime
sales['date'] = pd.to_datetime(sales['date'], format='%Y-%m-%d', errors='coerce')

# Check the data types of the DataFrame again
sales.dtypes

transaction_id            object
item                      object
quantity                   int32
price                    float64
spent                    float64
location                  object
date              datetime64[ns]
dtype: object

In [29]:
# Find the start and end dates of the data set
start_date = sales['date'].min()
end_date = sales['date'].max()

# Check the start and end dates of the data set
start_date, end_date

(Timestamp('2023-01-01 00:00:00'), Timestamp('2023-12-31 00:00:00'))

In [30]:
sales_null

,transaction_id,item,quantity,price,spent,location,date
77,TXN_2091733,Salad,1,5.0,5.0,In-store,NaN
104,TXN_7447872,Juice,2,3.0,6.0,In-store,NaN
160,TXN_1093800,Sandwich,3,4.0,12.0,Takeaway,NaN
175,TXN_6463132,Cookie,5,1.0,5.0,Takeaway,NaN
246,TXN_1908636,Tea,2,1.5,3.0,In-store,NaN
...,...,...,...,...,...,...,...
9766,TXN_7390866,Tea,5,1.5,7.5,Takeaway,NaN
9769,TXN_9686177,Cake,3,3.0,9.0,In-store,NaN
9833,TXN_5536245,Smoothie,4,4.0,16.0,In-store,NaN
9931,TXN_8344810,Smoothie,2,4.0,8.0,In-store,NaN


In [31]:
# Generate random dates between the start and end date in the date column of the sales_null DataFrame
sales_null['date'] = pd.to_datetime(
    np.random.choice(pd.date_range(start_date, end_date, freq='D'), size=len(sales_null))
)

# Generate random dates between the start and end date in the date column of the sales_error DataFrame
sales_error['date'] = pd.to_datetime(
    np.random.choice(pd.date_range(start_date, end_date, freq='D'), size=len(sales_error))
)

In [32]:
# Check the date column of the sales_null DataFrame
sales_null[['date']]

,date
77,2023-01-18
104,2023-01-23
160,2023-02-25
175,2023-03-03
246,2023-02-23
...,...
9766,2023-08-02
9769,2023-06-18
9833,2023-08-06
9931,2023-04-25


In [33]:
# Check the date column of the sales_error DataFrame
sales_error[['date']]

,date
11,2023-04-19
29,2023-06-07
33,2023-01-05
103,2023-12-17
115,2023-11-02
...,...
9907,2023-03-09
9933,2023-11-30
9937,2023-06-11
9949,2023-11-09


In [34]:
# heck the shape of the sales data before appending
sales.shape, sales_null.shape, sales_error.shape

((8634, 7), (142, 7), (271, 7))

In [35]:
# Append the sales_null and sales_error DataFrames to the sales DataFrame
sales = pd.concat([sales, sales_null, sales_error], ignore_index=True)

# Check the shape of the DataFrame again
sales.shape

(9047, 7)

In [36]:
# Create a new csv containing the cleaned sales data
sales.to_csv('cafe_sales_data_cleaned.csv', index=False)


There we have it. A fully cleaned sales data using a number of different methods in order to preserve as many rows of the data as possible. Now obviously there will be some implications of extrapolating out the location data and randomising the date data. But as these issues made up such a small part of the data, I believe that this data will be sufficient to accurately answer our initial questions and continue on to do a deeper analysis.